In [1]:
import importlib as il
import numpy as np
import h5py

from ase.calculators.emt import EMT
from ase.io import read
from hilde.helpers.supercell import make_cubic_supercell
from hilde.tasks.calculate import calculate_multiple
from hilde.structure.io import inform

In [2]:
ph3 = il.import_module('hilde.phono3py.phono3')

In [3]:
atoms = read('../Al.in', format='aims')

In [4]:
sc2, smatrix2 = make_cubic_supercell(atoms, 64)
sc3, smatrix3 = make_cubic_supercell(atoms, 30)
inform(sc3)


Geometry info for:
  input geometry:    Al32
  Number of atoms:   32
  Species:           Al
  Periodicity:       [ True  True  True]
  Lattice:  
    [8.094532 0.       0.      ]
    [0.       8.094532 0.      ]
    [0.       0.       8.094532]

  supercell_matrix: [-2, 2, 2, 2, -2, 2, 2, 2, -2]

Cell lengths and angles [Å, °]:
  a, b, c:      8.0945      8.0945      8.0945
  α, β, γ:     90.0000     90.0000     90.0000
  Volume:     530.3655 Å**3


In [5]:
phono3py_settings = {
    'atoms': atoms,
    'fc2_supercell_matrix': smatrix2,
    'fc3_supercell_matrix': smatrix3,
    'cutoff_pair_distance': 3
}

In [6]:
il.reload(ph3)
# help(ph3.preprocess)

<module 'hilde.phono3py.phono3' from '/home/knoop/local/hilde/hilde/phono3py/phono3.py'>

In [7]:
phonon3, sc2, sc3, scs2, scs3 = ph3.preprocess(**phono3py_settings)

In [8]:
print(f'Number of FC2 displacements: {len(scs2)}')
print(f'Number of FC3 displacements: {len(scs3)}')
print(f'Number of FC3 displacements that were created: {len([sc for sc in scs3 if sc])}')

Number of FC2 displacements: 2
Number of FC3 displacements: 37
Number of FC3 displacements that were created: 15


In [9]:
calc = EMT()

## Compute Forces

In [10]:
scs2_computed = calculate_multiple(scs2, calc, 'test_run/fc2', trajectory='fc2.traj')
fc2_forces = ph3.get_forces(scs2_computed)

In [11]:
%time scs3_computed = calculate_multiple(scs3, calc, 'test_run/fc3', trajectory='fc3.traj')
fc3_forces = ph3.get_forces(scs3_computed)
len(scs3_computed), len(scs3)

CPU times: user 520 ms, sys: 159 µs, total: 520 ms
Wall time: 518 ms


(37, 37)

In [12]:
[s for s in scs3_computed if s][-1].info

{'displacement_id': 28}

## Produce Force Constants

In [13]:
phonon3.produce_fc2(fc2_forces)

In [14]:
%time phonon3.produce_fc3(fc3_forces)

Computing fc3[ 1, x, x ] using numpy.linalg.pinv with a displacement:
    [ 0.0300  0.0000  0.0000]
Expanding fc3
Cutting-off fc3 (cut-off distance: 3.000000)
Building atom mapping table...
Creating contracted fc3...
CPU times: user 2.78 s, sys: 535 ms, total: 3.32 s
Wall time: 2.67 s


In [15]:
fc2 = phonon3.get_fc2()
fc3 = phonon3.get_fc3()

In [16]:
q_mesh = 3*[3]
phonon3_new = ph3.prepare_phono3py(**phono3py_settings,
                                   fc2=fc2,
                                   fc3=fc3,
                                   q_mesh=q_mesh,
                                   log_level=0)

In [17]:
phonon3_new.run_thermal_conductivity(write_kappa=True)

In [18]:
with h5py.File('kappa-m{}.hdf5'.format(''.join(map(str, q_mesh)))) as kappa:
    print(kappa['frequency'][:])
    print(kappa['kappa'][:][np.where(kappa['temperature'][:] == 300)][:3])
    assert 5 < kappa['frequency'][:].max() < 10
    # assert 25 < kappa['kappa'][:][30][:3].mean() < 35

[[-1.10904910e-06  1.94533367e-07  1.35757994e-06]
 [-1.06954566e+00  2.85642562e+00  5.36872093e+00]
 [-5.52775954e-01  4.59473925e+00  6.60728814e+00]
 [-7.76155124e-01  5.41591746e+00  6.86746735e+00]]
[[ 3.17337731e+01  3.17337731e+01  3.17337731e+01 -2.06432817e-16
  -2.42726872e-17 -7.28371948e-17]]
